In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("Aggregation"). \
        master("yarn"). \
        getOrCreate()

In [3]:
hotel_bookings_schema = "booking_id int, guest_name string, checkin_date date, checkout_date date, room_type string, total_price double"

In [4]:
hotel_bookings = spark.read.csv("/public/trendytech/datasets/hotel_data.csv", schema = hotel_bookings_schema)

In [5]:
hotel_bookings.show()

+----------+-----------------+------------+-------------+---------+-----------+
|booking_id|       guest_name|checkin_date|checkout_date|room_type|total_price|
+----------+-----------------+------------+-------------+---------+-----------+
|         1|         John Doe|  2023-05-01|   2023-05-05| Standard|      400.0|
|         2|       Jane Smith|  2023-05-02|   2023-05-06|   Deluxe|      600.0|
|         3|     Mark Johnson|  2023-05-03|   2023-05-08| Standard|      450.0|
|         4|     Sarah Wilson|  2023-05-04|   2023-05-07|Executive|      750.0|
|         5|      Emily Brown|  2023-05-06|   2023-05-09|   Deluxe|      550.0|
|         6|    Michael Davis|  2023-05-07|   2023-05-10| Standard|      400.0|
|         7|Samantha Thompson|  2023-05-08|   2023-05-12|   Deluxe|      600.0|
|         8|      William Lee|  2023-05-10|   2023-05-13| Standard|      450.0|
|         9|    Amanda Harris|  2023-05-11|   2023-05-16|Executive|      750.0|
|        10|  David Rodriguez|  2023-05-

In [6]:
hotel_bookings.printSchema()

root
 |-- booking_id: integer (nullable = true)
 |-- guest_name: string (nullable = true)
 |-- checkin_date: date (nullable = true)
 |-- checkout_date: date (nullable = true)
 |-- room_type: string (nullable = true)
 |-- total_price: double (nullable = true)



### 1. Average Order Value of 

In [7]:
from pyspark.sql.functions import avg, round, expr, date_sub, datediff, month, count, sum

In [8]:
from pyspark.sql import Window

In [9]:
x = hotel_bookings.agg(round(avg("total_price"),0).alias("average_order_value"))

In [10]:
x

average_order_value
546.0


In [11]:
type(x)

pyspark.sql.dataframe.DataFrame

In [12]:
y = x.collect()

In [13]:
type(y)

list

In [14]:
print("{} | {} | {} | {}".format(y,y[0],type(y[0]),y[0][0]))

[Row(average_order_value=546.0)] | Row(average_order_value=546.0) | <class 'pyspark.sql.types.Row'> | 546.0


In [15]:
print("Average order values of the hotel bookings is {} Rs.".format(x.collect()[0][0]))

Average order values of the hotel bookings is 546.0 Rs.


### 2. Average Rental Time

In [16]:
hb_with_duration = hotel_bookings. \
                    withColumn("duration_of_stay",datediff(hotel_bookings.checkout_date, hotel_bookings.checkin_date))

In [17]:
hb_with_duration.show()

+----------+-----------------+------------+-------------+---------+-----------+----------------+
|booking_id|       guest_name|checkin_date|checkout_date|room_type|total_price|duration_of_stay|
+----------+-----------------+------------+-------------+---------+-----------+----------------+
|         1|         John Doe|  2023-05-01|   2023-05-05| Standard|      400.0|               4|
|         2|       Jane Smith|  2023-05-02|   2023-05-06|   Deluxe|      600.0|               4|
|         3|     Mark Johnson|  2023-05-03|   2023-05-08| Standard|      450.0|               5|
|         4|     Sarah Wilson|  2023-05-04|   2023-05-07|Executive|      750.0|               3|
|         5|      Emily Brown|  2023-05-06|   2023-05-09|   Deluxe|      550.0|               3|
|         6|    Michael Davis|  2023-05-07|   2023-05-10| Standard|      400.0|               3|
|         7|Samantha Thompson|  2023-05-08|   2023-05-12|   Deluxe|      600.0|               4|
|         8|      William Lee|

In [18]:
avg_stay_duration = hb_with_duration.agg(round(avg("duration_of_stay"),0).alias("avg_dur_of_stay"))

In [19]:
print("Average duration of stay is {} days". \
      format(int(avg_stay_duration.collect()[0][0]))
     )

# Collect will return a pyspark.sql.Rowspark 

Average duration of stay is 8 days


### 3. Monthly total amount and total bookings for each room type

In [20]:
hb_with_month = hb_with_duration.withColumn("booking_month",month("checkin_date"))

In [21]:
monthly_agg = hb_with_month.groupBy("room_type","booking_month") \
              .agg(
                   count("*").alias("total_bookings"), 
                   sum("total_price").alias("total_amount")
                 ) \
              .orderBy("room_type","booking_month")

In [22]:
monthly_agg.show()

+---------+-------------+--------------+------------+
|room_type|booking_month|total_bookings|total_amount|
+---------+-------------+--------------+------------+
|   Deluxe|            5|             9|      5200.0|
|   Deluxe|            6|            10|      5750.0|
|   Deluxe|            7|            16|      9200.0|
|   Deluxe|            8|             8|      4600.0|
|Executive|            5|             5|      3750.0|
|Executive|            6|             5|      3750.0|
|Executive|            7|             6|      4500.0|
|Executive|            8|             4|      3000.0|
| Standard|            5|            10|      4250.0|
| Standard|            6|            10|      4250.0|
| Standard|            7|            16|      6800.0|
| Standard|            8|             8|      3400.0|
+---------+-------------+--------------+------------+



In [23]:
hb_with_month.groupBy("room_type") \
              .agg(
                   count("*").alias("total_bookings"), 
                   sum("total_price").alias("total_amount")
                 ) \
              .orderBy("room_type") \
              .show()

+---------+--------------+------------+
|room_type|total_bookings|total_amount|
+---------+--------------+------------+
|   Deluxe|            43|     24750.0|
|Executive|            20|     15000.0|
| Standard|            44|     18700.0|
+---------+--------------+------------+



### 4. Monthly running total amount and running total bookings for each room type

In [24]:
window = Window.partitionBy("room_type") \
                .orderBy("booking_month") \
                .rowsBetween(Window.unboundedPreceding, Window.currentRow)

In [25]:
monthly_agg.withColumn( "total_amount_cumulative",
                           sum("total_amount").over(window)
                         ) \
            .withColumn(
                         "total_bookings_cumulative",
                         sum("total_bookings").over(window)
                        ) \
            .show()

+---------+-------------+--------------+------------+-----------------------+-------------------------+
|room_type|booking_month|total_bookings|total_amount|total_amount_cumulative|total_bookings_cumulative|
+---------+-------------+--------------+------------+-----------------------+-------------------------+
|Executive|            5|             5|      3750.0|                 3750.0|                        5|
|Executive|            6|             5|      3750.0|                 7500.0|                       10|
|Executive|            7|             6|      4500.0|                12000.0|                       16|
|Executive|            8|             4|      3000.0|                15000.0|                       20|
|   Deluxe|            5|             9|      5200.0|                 5200.0|                        9|
|   Deluxe|            6|            10|      5750.0|                10950.0|                       19|
|   Deluxe|            7|            16|      9200.0|           

### 5. Ranking Room Type by monthly booking amount

In [26]:
monthly_agg.show()

+---------+-------------+--------------+------------+
|room_type|booking_month|total_bookings|total_amount|
+---------+-------------+--------------+------------+
|   Deluxe|            5|             9|      5200.0|
|   Deluxe|            6|            10|      5750.0|
|   Deluxe|            7|            16|      9200.0|
|   Deluxe|            8|             8|      4600.0|
|Executive|            5|             5|      3750.0|
|Executive|            6|             5|      3750.0|
|Executive|            7|             6|      4500.0|
|Executive|            8|             4|      3000.0|
| Standard|            5|            10|      4250.0|
| Standard|            6|            10|      4250.0|
| Standard|            7|            16|      6800.0|
| Standard|            8|             8|      3400.0|
+---------+-------------+--------------+------------+



In [27]:
from pyspark.sql.functions import rank, row_number, dense_rank, lag, lead, when, coalesce, isnull, lit, col

In [28]:
window2 = Window.partitionBy("room_type") \
                .orderBy("total_amount")

In [29]:
monthly_agg.withColumn( "rank", 
                        rank().over(window2)
                      ) \
            .withColumn( "dense_rank", 
                        dense_rank().over(window2)
                      ) \
            .withColumn( "row_number", 
                        row_number().over(window2)
                      ) \
            .show()

+---------+-------------+--------------+------------+----+----------+----------+
|room_type|booking_month|total_bookings|total_amount|rank|dense_rank|row_number|
+---------+-------------+--------------+------------+----+----------+----------+
|Executive|            8|             4|      3000.0|   1|         1|         1|
|Executive|            5|             5|      3750.0|   2|         2|         2|
|Executive|            6|             5|      3750.0|   2|         2|         3|
|Executive|            7|             6|      4500.0|   4|         3|         4|
|   Deluxe|            8|             8|      4600.0|   1|         1|         1|
|   Deluxe|            5|             9|      5200.0|   2|         2|         2|
|   Deluxe|            6|            10|      5750.0|   3|         3|         3|
|   Deluxe|            7|            16|      9200.0|   4|         4|         4|
| Standard|            8|             8|      3400.0|   1|         1|         1|
| Standard|            5|   

### 6. Calculate the Month on Month Growth or Decline for each room type

In [30]:
window3 = Window.partitionBy("room_type") \
                .orderBy("booking_month")

In [31]:
monthly_agg_2 = monthly_agg.withColumnRenamed("total_amount","current_month_amount") \
                            .withColumn("previous_month_amount",
                                         lag("current_month_amount").over(window3)
                                       )

In [32]:
monthly_agg_2.show()

+---------+-------------+--------------+--------------------+---------------------+
|room_type|booking_month|total_bookings|current_month_amount|previous_month_amount|
+---------+-------------+--------------+--------------------+---------------------+
|Executive|            5|             5|              3750.0|                 null|
|Executive|            6|             5|              3750.0|               3750.0|
|Executive|            7|             6|              4500.0|               3750.0|
|Executive|            8|             4|              3000.0|               4500.0|
|   Deluxe|            5|             9|              5200.0|                 null|
|   Deluxe|            6|            10|              5750.0|               5200.0|
|   Deluxe|            7|            16|              9200.0|               5750.0|
|   Deluxe|            8|             8|              4600.0|               9200.0|
| Standard|            5|            10|              4250.0|               

In [33]:
monthly_agg_2.printSchema()

root
 |-- room_type: string (nullable = true)
 |-- booking_month: integer (nullable = true)
 |-- total_bookings: long (nullable = false)
 |-- current_month_amount: double (nullable = true)
 |-- previous_month_amount: double (nullable = true)



In [39]:
monthly_agg_2.withColumn( "% Growth/Decline",
                         when(isnull("previous_month_amount"), "N/A")
                         .otherwise(expr(
                                            """round(
                                                        ((current_month_amount - previous_month_amount) 
                                                           / previous_month_amount * 100)
                                                     ,2)"""
                                         )
                                   )
                        ) \
            .withColumn( "performance",
                         when(col("% Growth/Decline") > 0, "Growth")
                         .when(col("% Growth/Decline") < 0, "Decline")
                         .when(col("% Growth/Decline") == 0, "Same as Prev Month")
                         .otherwise("First Month")
                       ) \
            .show()

+---------+-------------+--------------+--------------------+---------------------+----------------+------------------+
|room_type|booking_month|total_bookings|current_month_amount|previous_month_amount|% Growth/Decline|       performance|
+---------+-------------+--------------+--------------------+---------------------+----------------+------------------+
|Executive|            5|             5|              3750.0|                 null|             N/A|       First Month|
|Executive|            6|             5|              3750.0|               3750.0|             0.0|Same as Prev Month|
|Executive|            7|             6|              4500.0|               3750.0|            20.0|            Growth|
|Executive|            8|             4|              3000.0|               4500.0|          -33.33|           Decline|
|   Deluxe|            5|             9|              5200.0|                 null|             N/A|       First Month|
|   Deluxe|            6|            10|

### 7. Using the input data create a Pivot Table with Room Type as Rows and Month as Cols and Total Amount as values

In [35]:
hb_with_month.show()

+----------+-----------------+------------+-------------+---------+-----------+----------------+-------------+
|booking_id|       guest_name|checkin_date|checkout_date|room_type|total_price|duration_of_stay|booking_month|
+----------+-----------------+------------+-------------+---------+-----------+----------------+-------------+
|         1|         John Doe|  2023-05-01|   2023-05-05| Standard|      400.0|               4|            5|
|         2|       Jane Smith|  2023-05-02|   2023-05-06|   Deluxe|      600.0|               4|            5|
|         3|     Mark Johnson|  2023-05-03|   2023-05-08| Standard|      450.0|               5|            5|
|         4|     Sarah Wilson|  2023-05-04|   2023-05-07|Executive|      750.0|               3|            5|
|         5|      Emily Brown|  2023-05-06|   2023-05-09|   Deluxe|      550.0|               3|            5|
|         6|    Michael Davis|  2023-05-07|   2023-05-10| Standard|      400.0|               3|            5|
|

In [36]:
# It is a best practice to define the column headers before running pivot.
# It helps in reducing the overhead of scanning the pivot column values before running the pivot

month_values = [5,6,7,8]

In [42]:
hb_with_month.groupBy("room_type") \
             .pivot("booking_month",month_values) \
             .agg(sum("total_price")) \
             .show()

+---------+------+------+------+------+
|room_type|     5|     6|     7|     8|
+---------+------+------+------+------+
|Executive|3750.0|3750.0|4500.0|3000.0|
|   Deluxe|5200.0|5750.0|9200.0|4600.0|
| Standard|4250.0|4250.0|6800.0|3400.0|
+---------+------+------+------+------+



In [43]:
spark.stop()